In [4]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set model storage path inside Google Drive
mistral_path = "/content/drive/MyDrive/mistral_models/7B-Instruct-v0.3"

# Create the directory if it doesn't exist
os.makedirs(mistral_path, exist_ok=True)

print(f"Model directory set: {mistral_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model directory set: /content/drive/MyDrive/mistral_models/7B-Instruct-v0.3


In [5]:
!pip install mistral_inference

In [6]:
# import shutil

# # Rename model file
# old_model_path = os.path.join(mistral_path, "consolidated.safetensors")
# new_model_path = os.path.join(mistral_path, "model.safetensors")

# if os.path.exists(old_model_path):
#     shutil.move(old_model_path, new_model_path)
#     print("Model file renamed!")

# # Rename tokenizer file
# old_tokenizer_path = os.path.join(mistral_path, "tokenizer.model.v3")
# new_tokenizer_path = os.path.join(mistral_path, "tokenizer.model")

# if os.path.exists(old_tokenizer_path):
#     shutil.move(old_tokenizer_path, new_tokenizer_path)
#     print("Tokenizer file renamed!")

In [7]:
!pip install transformers accelerate sentencepiece torch safetensors

In [8]:
!pip install bitsandbytes

###FP16 (Half Precision, 2 bytes per parameter) takes lot of ram so i am sifting to int4

In [9]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# mistral_path = "mistralai/Mistral-7B-Instruct-v0.3"

# # Configure 4-bit quantization
# bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(mistral_path, use_fast=False)

# # Load model with quantization
# model = AutoModelForCausalLM.from_pretrained(
#     mistral_path,
#     quantization_config=bnb_config,  # Enables 4-bit quantization
#     device_map="auto",  # Automatically assigns model to available GPU(s)
# )

# print("Model and Tokenizer Loaded Successfully (4-bit Quantized)!")

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model and Tokenizer Loaded Successfully (4-bit Quantized)!


In [12]:
# Move model to correct device
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# # Example input
# input_text = "write an eassay on What is the meaning of life?"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# # Generate response
# output = model.generate(input_ids, max_length=100)
# response = tokenizer.decode(output[0], skip_special_tokens=True)

# print("\nGenerated Response:")
# print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generated Response:
write an eassay on What is the meaning of life?

Title: The Meaning of Life: A Philosophical and Existential Inquiry

Introduction

The question of the meaning of life is one of the most profound and enduring inquiries in human history, transcending the boundaries of culture, religion, and philosophy. This essay aims to explore the meaning of life from a philosophical and existential perspective, delving into the various theories and interpretations


In [14]:
# input_text = "i am kalam write an eassay 100 words"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# # Generate response
# output = model.generate(input_ids, max_length=100)
# response = tokenizer.decode(output[0], skip_special_tokens=True)

# print("\nGenerated Response:")
# print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generated Response:
i am kalam write an eassay 100 words on the importance of education

Education is the cornerstone of human development, empowering individuals and communities to break the cycle of poverty and achieve their full potential. It equips us with the knowledge, skills, and values necessary to navigate the complexities of the modern world, fostering critical thinking, creativity, and empathy. Education is a powerful tool for social and economic mobility, enabling individuals to access


In [15]:
# input_text = "i am kalam write an eassay 100 words continue from Education is a powerful tool for social and economic mobility, enabling individuals to access"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# # Generate response
# output = model.generate(input_ids, max_length=100)
# response = tokenizer.decode(output[0], skip_special_tokens=True)

# print("\nGenerated Response:")
# print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generated Response:
i am kalam write an eassay 100 words continue from Education is a powerful tool for social and economic mobility, enabling individuals to access opportunities and improve their lives. Education provides knowledge, skills, and values that are essential for personal development and for contributing to society. It fosters critical thinking, creativity, and problem-solving abilities, empowering individuals to make informed decisions and to navigate complex challenges. Education also promotes social cohesion, cultural understanding, and


###shift to 8bit quantization

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

mistral_path = "mistralai/Mistral-7B-Instruct-v0.3"

# Configure 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(mistral_path, use_fast=False)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    mistral_path,
    quantization_config=bnb_config,  # Enables 8-bit quantization
    device_map="auto",  # Automatically assigns model to available GPU(s)
)

print("Model and Tokenizer Loaded Successfully (8-bit Quantized)!")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model and Tokenizer Loaded Successfully (8-bit Quantized)!


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Example input
input_text = "What is the meaning of life?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# Generate response
output = model.generate(input_ids, max_length=100)
response = tokenizer.decode(output[0], skip_special_tokens=True)

print("\nGenerated Response:")
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generated Response:
What is the meaning of life?

This is a question that has puzzled philosophers, theologians, and scientists for centuries. Some people believe that the meaning of life is to seek happiness, while others believe that it is to serve a higher purpose or to fulfill a specific destiny.

One possible answer to this question is that the meaning of life is whatever we make it. Each person has the power to create their own purpose and to find meaning in their own way


In [19]:
while(True):
  input_text = input()
  if(input_text=="X"):
    break
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
  output = model.generate(input_ids, max_length=200)
  response = tokenizer.decode(output[0], skip_special_tokens=True)
  print("\nGenerated Response:")
  print(response)


write an essay on kalam


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generated Response:
write an essay on kalam cosmological argument

Title: The Kalam Cosmological Argument: A Rational Approach to the Existence of God

Introduction

The Kalam Cosmological Argument (KCA) is a philosophical argument for the existence of God, primarily associated with the works of the Muslim philosopher Al-Ghazali and the Jewish philosopher Maimonides, but later developed and popularized by Christian philosophers such as William Lane Craig. The KCA is a deductive argument that aims to demonstrate the necessity of a transcendent, uncaused, and personal cause for the existence of the universe.

Premise 1: Everything that begins to exist has a cause.

This premise is based on common sense and empirical evidence. In our everyday experience, we observe that everything that begins to exist has a cause. For example, a house is built by a builder, a painting is
write an essay on APJ abdul kalam


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generated Response:
write an essay on APJ abdul kalam

APJ Abdul Kalam, popularly known as the Missile Man of India, was born on October 15, 1931, in Rameswaram, Tamil Nadu. He was the 11th President of India, serving from 2002 to 2007. Kalam was a scientist, aerospace engineer, and a teacher who dedicated his life to the service of the nation.

Kalam's early life was marked by hardship and determination. He lost his father at a young age and was raised by his mother. Despite the financial constraints, Kalam's thirst for knowledge was insatiable. He completed his schooling in Ramanathapuram and moved to Madras (now Chennai) for higher education. He graduated from the Madras Institute of Technology in 1954
X
